In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
import sys

In [ ]:
print(os.listdir('../input/folder/folder'))

In [ ]:
im=Image.open('../input/dataset/train/100.jpg')
plt.imshow(im)

In [ ]:
im=Image.open('../input/dataset/train/10.jpg')
plt.imshow(im)

In [ ]:
!git clone https://github.com/Tessellate-Imaging/monk_v1.git
!cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

In [ ]:
sys.path.append("/kaggle/working/monk_v1/monk/");
from gluon_prototype import prototype
gtf = prototype(verbose=1);
gtf.Prototype("Dance_Form", "Using-resnet152_v1");
gtf.Default(dataset_path="../input/final3/folder/", 
            model_name="resnet152_v1", 
            freeze_base_network=False,
            num_epochs=10);

gtf.Train();

In [ ]:
os.listdir('../input/dance-form/dataset/test')

# For saving the predicted images

In [ ]:
list_test=os.listdir("../input/data-test/test/");
from tqdm.notebook import tqdm
combined = [];
for i in tqdm(range(len(list_test))):
    img_name = "../input/data-test/test/" + list_test[i];
    predictions = gtf.Infer(img_name=img_name);
    id_ = list_test[i]
    label = predictions["predicted_class"];    
    combined.append([id_, label]);

In [ ]:
a = pd.DataFrame(combined, columns = ['Image', 'target']);
a.to_csv("submission.csv", index=False);

# For Data augmentation

In [ ]:
from imgaug import augmenters as iaa
import imgaug as ia
import imageio
seq=iaa.Sequential([
        iaa.Scale((224, 224)),
        iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),
        iaa.Fliplr(0.5),
        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
        iaa.Affine(rotate=(-20, 20), mode='symmetric'),
        iaa.Sometimes(0.25,
                      iaa.OneOf([iaa.Dropout(p=(0, 0.1)),
                                 iaa.CoarseDropout(0.1,                                                                 size_percent=0.5)])),
        iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)
    ])

# Saving the augmented images

In [ ]:
path='/data/folder/'

In [ ]:
for folder in os.listdir(path):
    i = 0
    for fname in os.listdir(path + '/' + folder):
        img = imageio.imread(path + '/' + folder + '/' + fname)
        print('Original:')
        ia.imshow(img)
        img_aug = seq.augment_image(img)
        print('Augmented:')
        ia.imshow(img_aug)
        
        imageio.imwrite(os.path.join(path, path + '/' + folder + '/' + folder + "%06d.jpg" % (i,)), img_aug)
        i += 1